In [1]:
import polars as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics.pairwise import cosine_distances, euclidean_distances, manhattan_distances
import warnings
from statsmodels.stats.multitest import multipletests



In [2]:
atac_data_df = pd.read_csv('../../data/embryo/raw/atac_peak_matrix_complete_sparse.csv', index_col=0)




In [3]:
rna_data = pl.read_csv("../../data/embryo/raw/scRNA_countMatrix.csv", ignore_errors=True)
# Convert to dictionary first, then to pandas
data_dict = rna_data.to_dict(as_series=False)
rna_data_df = pd.DataFrame(data_dict)
# make 1st colun the index 
rna_data_df.set_index(rna_data_df.columns[0], inplace=True)
# remove empty rows and columns and set gene names as index
rna_data_df[rna_data_df.isna().any(axis=1)]
rna_data_df.dropna(axis=0, how='any', inplace=True)

rna_data_log = rna_data_df.copy()
rna_data_log.iloc[:, 0:] = np.log2(rna_data_log.iloc[:, 0:].astype(float) + 1)
rna_data_log


,10h-1_CELL1140_N1_10h-1,10h-1_CELL1447_N1_10h-1,10h-1_CELL1347_N1_10h-1,10h-1_CELL1204_N1_10h-1,10h-1_CELL97_N2_10h-1,10h-1_CELL539_N1_10h-1,10h-1_CELL310_N1_10h-1,10h-1_CELL311_N1_10h-1,10h-1_CELL665_N1_10h-1,10h-1_CELL350_N1_10h-1,...,6h_3 CELL3882_N1 _,6h_3 CELL5301_N1 _,6h_3 CELL5240_N1 _,6h_3 CELL5143_N1 _,6h_3 CELL5575_N1 _,6h_3 CELL5349_N1 _,6h_3 CELL4434_N1 _,6h_3 CELL5403_N1 _,6h_3 CELL5489_N1 _,6h_3 CELL5447_N1 _
,,,,,,,,,,,,,,,,,,,,,
rpl13a,5.554589,4.523562,4.807355,4.247928,6.209453,6.228819,6.507795,6.599913,5.882643,5.754888,...,2.321928,0.000000,1.584963,2.000000,1.584963,1.584963,2.000000,3.000000,1.000000,1.000000
khdrbs1a,6.357552,5.906891,5.209453,5.930737,7.294621,7.375039,7.531381,7.599913,6.894818,7.228819,...,5.459432,4.906891,4.459432,3.906891,4.584963,4.584963,5.129283,4.584963,4.807355,5.700440
apoeb,6.614710,6.870365,7.599913,4.700440,4.754888,9.144658,6.554589,5.554589,6.569856,7.149747,...,3.584963,3.000000,3.321928,4.087463,4.321928,3.459432,4.321928,2.321928,1.584963,1.000000
cfl1,3.584963,3.321928,3.459432,4.459432,4.954196,5.247928,5.614710,6.044394,4.906891,5.392317,...,3.169925,2.000000,1.584963,1.000000,2.000000,1.584963,3.169925,2.584963,2.000000,2.807355
polr2d,2.584963,1.000000,1.000000,2.807355,2.000000,2.000000,2.584963,2.807355,1.000000,3.000000,...,0.000000,0.000000,0.000000,1.584963,0.000000,0.000000,1.000000,1.000000,1.000000,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CU570797.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
trgv5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
BX950188.3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [4]:
rna_data_log_filtered_low_genes = rna_data_log[rna_data_log.max(axis=1) > 4]
rna_data_log_filtered_low_genes # keeps ~9000 genes out of 20000


,10h-1_CELL1140_N1_10h-1,10h-1_CELL1447_N1_10h-1,10h-1_CELL1347_N1_10h-1,10h-1_CELL1204_N1_10h-1,10h-1_CELL97_N2_10h-1,10h-1_CELL539_N1_10h-1,10h-1_CELL310_N1_10h-1,10h-1_CELL311_N1_10h-1,10h-1_CELL665_N1_10h-1,10h-1_CELL350_N1_10h-1,...,6h_3 CELL3882_N1 _,6h_3 CELL5301_N1 _,6h_3 CELL5240_N1 _,6h_3 CELL5143_N1 _,6h_3 CELL5575_N1 _,6h_3 CELL5349_N1 _,6h_3 CELL4434_N1 _,6h_3 CELL5403_N1 _,6h_3 CELL5489_N1 _,6h_3 CELL5447_N1 _
,,,,,,,,,,,,,,,,,,,,,
rpl13a,5.554589,4.523562,4.807355,4.247928,6.209453,6.228819,6.507795,6.599913,5.882643,5.754888,...,2.321928,0.000000,1.584963,2.000000,1.584963,1.584963,2.000000,3.000000,1.000000,1.000000
khdrbs1a,6.357552,5.906891,5.209453,5.930737,7.294621,7.375039,7.531381,7.599913,6.894818,7.228819,...,5.459432,4.906891,4.459432,3.906891,4.584963,4.584963,5.129283,4.584963,4.807355,5.700440
apoeb,6.614710,6.870365,7.599913,4.700440,4.754888,9.144658,6.554589,5.554589,6.569856,7.149747,...,3.584963,3.000000,3.321928,4.087463,4.321928,3.459432,4.321928,2.321928,1.584963,1.000000
cfl1,3.584963,3.321928,3.459432,4.459432,4.954196,5.247928,5.614710,6.044394,4.906891,5.392317,...,3.169925,2.000000,1.584963,1.000000,2.000000,1.584963,3.169925,2.584963,2.000000,2.807355
polr2d,2.584963,1.000000,1.000000,2.807355,2.000000,2.000000,2.584963,2.807355,1.000000,3.000000,...,0.000000,0.000000,0.000000,1.584963,0.000000,0.000000,1.000000,1.000000,1.000000,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tmc2a,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
gnb3a,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
cd164l2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [5]:
atac_metadata_df = pd.read_csv('../../data/embryo/raw/atac_all.metaData.txt', index_col=0)
atac_metadata_df = atac_metadata_df[['celltype', 'predictedCell', 'predictedGroup'] ]
atac_metadata_df.rename(columns = {'celltype': 'atac_cell_type', 'predictedCell': 'rna_matching_cell', 'predictedGroup': 'rna_cell_type'}, inplace=True)
atac_metadata_df.reset_index(names = "atac_cell", inplace=True)
atac_metadata_df.head()


,atac_cell,atac_cell_type,rna_matching_cell,rna_cell_type
0,3hpf_1#3hpf_1_merge_BC0443_N27,blastomere,6h_3 CELL4645_N1 _,margin
1,3hpf_1#3hpf_1_merge_BC0069_N07,blastomere,3h1_CELL1337_N1_3h1,blastomere
2,3hpf_1#3hpf_1_merge_BC0033_N05,blastomere,3h1_CELL1337_N1_3h1,blastomere
3,3hpf_1#3hpf_1_merge_BC0028_N08,blastomere,3h1_CELL1197_N1_3h1,blastomere
4,3hpf_1#3hpf_1_merge_BC0264_N08,blastomere,3h1_CELL1197_N1_3h1,blastomere


In [6]:
rna_metadata_df = pd.read_csv('../../data/embryo/raw/rna_all.metaData.txt')
rna_metadata_df.rename(columns = {'Unnamed: 0': 'rna_cell'}, inplace=True)
rna_metadata_df


,rna_cell,orig.ident,nCount_RNA,nFeature_RNA,percent.mt,seurat_clusters,stage,celltype
0,10h-1_CELL1140_N1_10h-1,10h-1,16009,3354,1.467924,14,10hpf,anterior neural keel
1,10h-1_CELL1447_N1_10h-1,10h-1,11259,2505,1.367795,3,10hpf,neurogenic placode
2,10h-1_CELL1347_N1_10h-1,10h-1,12532,2806,1.324609,22,10hpf,epithelial Cell
3,10h-1_CELL1204_N1_10h-1,10h-1,14432,3312,1.489745,8,10hpf,tail bud
4,10h-1_CELL97_N2_10h-1,10h-1,31455,4443,1.176284,2,10hpf,posterior neural keel
...,...,...,...,...,...,...,...,...
68848,6h_3 CELL5349_N1 _,6hpf_3,2708,1097,1.289134,1,6hpf,epiblast
68849,6h_3 CELL4434_N1 _,6hpf_3,3852,1492,1.011936,1,6hpf,epiblast
68850,6h_3 CELL5403_N1 _,6hpf_3,2694,1098,0.667904,1,6hpf,epiblast
68851,6h_3 CELL5489_N1 _,6hpf_3,2602,972,1.345119,1,6hpf,epiblast


In [7]:
rna_metadata_df['stage_celltype'] = rna_metadata_df['stage'].str.extract(r'(\d{1,2})').astype(str) + '_' + rna_metadata_df['celltype']
rna_cell_to_pseudobulk = dict(zip(rna_metadata_df['rna_cell'], rna_metadata_df['stage_celltype']))

rna_data_filtered = rna_data_df[rna_data_df.columns.intersection(rna_cell_to_pseudobulk.keys())]
rna_data_filtered.columns = [rna_cell_to_pseudobulk[cell] for cell in rna_data_filtered.columns]

rna_mean_psd = rna_data_filtered.groupby(axis=1, by=rna_data_filtered.columns).mean()

print("Shape of RNA mean pseudobulk matrix:", rna_mean_psd.shape)
rna_mean_psd.head()


ValueError: Cannot set a DataFrame with multiple columns to the single column stage_celltype

In [ ]:
celltype_lookup = dict(zip(atac_metadata_df['atac_cell'], atac_metadata_df['atac_cell_type'])) # map the cell types to the atac_cell names
atac_data_df['stage'] = atac_data_df['Cell'].str.extract(r'(\d{1,2})[a-zA-Z]', expand=False)
atac_data_df['celltype'] = atac_data_df['Cell'].map(celltype_lookup)
atac_data_df['stage_celltype'] = atac_data_df['stage'] + '_' + atac_data_df['celltype']
atac_stage_counts = atac_data_df.groupby('stage')['Cell'].nunique()
atac_celltype_counts = atac_data_df.groupby('celltype')['Cell'].nunique()
print("shape of atac_data_df:", atac_data_df.shape)
print(atac_data_df.head())
# create mean dataframe
atac_mean_psd  = atac_data_df.pivot_table(index='Peak', 
    columns='stage_celltype', values='Accessibility', aggfunc='mean')

# Create std dataframe
atac_std_psd = atac_data_df.pivot_table(index='Peak', 
    columns='stage_celltype', values='Accessibility', aggfunc='std')

# replace nan with 0
atac_mean_psd.fillna(0, inplace=True)
atac_mean_psd


shape of atac_data_df: (244265812, 5)
                                         Cell  Accessibility stage celltype  \
Peak                                                                          
chr1:12474-12974  24hpf_1#24hpf_1_BC00224_N06              1    24      UND   
chr1:14704-15204  24hpf_1#24hpf_1_BC00224_N06              1    24      UND   
chr1:16672-17172  24hpf_1#24hpf_1_BC00224_N06              3    24      UND   
chr1:18404-18904  24hpf_1#24hpf_1_BC00224_N06              3    24      UND   
chr1:19206-19706  24hpf_1#24hpf_1_BC00224_N06              2    24      UND   

                 stage_celltype  
Peak                             
chr1:12474-12974         24_UND  
chr1:14704-15204         24_UND  
chr1:16672-17172         24_UND  
chr1:18404-18904         24_UND  
chr1:19206-19706         24_UND  


stage_celltype,10_UND,10_YSL,10_anterior/posterior axis,10_lateral plate mesoderm,10_mesenchyme cell,10_neural crest,10_neural keel,10_periderm/epidermis,10_segmental plate,12_UND,...,5_EVL,5_YSL/presumptive endoderm,5_blastomere,5_epiblast,5_hypoblast,6_EVL,6_YSL/presumptive endoderm,6_blastomere,6_epiblast,6_hypoblast
Peak,,,,,,,,,,,,,,,,,,,,,
chr10:10002124-10002624,2.0,2.000000,1.800000,2.000000,0.0,2.000000,2.166667,4.000000,2.000000,0.0,...,1.571429,3.000000,0.0,2.000000,2.800000,1.333333,1.000000,0.0,2.200000,2.500000
chr10:10003707-10004207,1.0,0.000000,2.000000,0.000000,0.0,2.000000,1.888889,0.000000,1.666667,0.0,...,1.545455,3.000000,0.0,1.960000,2.375000,0.000000,0.000000,0.0,1.500000,0.000000
chr10:10004747-10005247,2.0,0.000000,1.750000,2.333333,0.0,2.285714,2.153846,1.666667,1.666667,2.0,...,1.666667,1.750000,0.0,1.642857,1.916667,2.500000,1.000000,0.0,1.400000,1.000000
chr10:10008047-10008547,2.0,0.000000,2.060606,2.250000,0.0,2.200000,2.116279,2.000000,2.250000,2.0,...,1.966667,0.000000,0.0,2.000000,2.076923,2.000000,2.000000,0.0,1.929825,2.333333
chr10:10009662-10010162,0.0,0.000000,1.800000,0.000000,0.0,1.750000,2.000000,1.500000,3.000000,0.0,...,1.600000,1.000000,0.0,2.000000,1.500000,1.000000,0.000000,0.0,2.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chr9:998557-999057,0.0,1.000000,2.000000,1.000000,0.0,1.500000,2.000000,0.000000,2.166667,0.0,...,2.000000,1.750000,0.0,2.172414,1.923077,3.000000,1.600000,0.0,1.900000,2.500000
chr9:9989556-9990056,2.0,1.958333,1.500000,2.000000,0.0,1.500000,2.750000,2.000000,2.000000,0.0,...,2.000000,1.833333,0.0,1.785714,2.000000,2.000000,2.705882,0.0,1.333333,1.000000
chr9:9992990-9993490,2.0,1.333333,1.000000,2.000000,0.0,0.000000,1.000000,1.000000,0.000000,0.0,...,1.000000,2.000000,0.0,1.692308,2.000000,1.000000,0.000000,0.0,2.000000,2.000000
